In [1]:
from pystac_client import Client
import pystac
import geopandas as gpd
import json
import netrc
import requests
from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
usgs_stac_url = 'https://landsatlook.usgs.gov/stac-server'
catalog = Client.open(usgs_stac_url)

In [3]:
df = gpd.read_file('../data/shapefiles/lake_mead/aoi.shp')
shape_geojson = json.loads(df.to_json())['features'][0]['geometry']

In [4]:
shape_geojson

{'type': 'Polygon',
 'coordinates': [[[-114.87222049842939, 36.16273394123082],
   [-114.50459507920661, 36.22948555290927],
   [-114.50643320630273, 36.53876281932189],
   [-114.02668203421706, 36.54023965625615],
   [-114.02852016131315, 35.975526918105764],
   [-114.86670611714104, 36.00973286261627],
   [-114.87222049842939, 36.16273394123082]]]}

In [5]:
opts = {
    'intersects' : shape_geojson,
    'collections':'landsat-c2l3-dswe',
    'datetime' : "2015-12-01/2020-12-31",
    'max_items' : 10
}

search = catalog.search(**opts)
items = search.get_all_items()
len(items)

10

In [9]:
print(items[0].assets.keys())

dict_keys(['index', 'diag', 'intr', 'intsm', 'inwam', 'mask', 'shade', 'xml', 'json'])


Straight up using code from [here](https://gist.github.com/scottyhq/ae2a53ecc955c5957c27c35d3bd9bce2) to download data from USGS

In [6]:
def download_asset(item:pystac.item.Item, asset_type:str='diag', download_path:str='../data/downloads'):
    download_path = Path(download_path)
    download_path.mkdir(exist_ok = True)

    asset_url = item.assets[asset_type.lower()].href
    filename = asset_url.split('/')[-1]
    
    creds = netrc.netrc()
    user,account,password = creds.authenticators('ers.cr.usgs.gov')

    url = 'https://ers.cr.usgs.gov/login'
    with requests.Session() as s:
        
        r = s.get(url)
        soup = BeautifulSoup(r.content, 'html.parser') 
        sval = soup.find('input', attrs={'name':'csrf'})['value']

        data = {"username": user, 
            "password": password,
            "csrf": sval}

        print('Authenticating with netrc...')
        bf = s.post(url, data = data)
        print(bf.status_code)

    print(f'Downloading {filename}...')
    respb = s.get(asset_url,
                allow_redirects=True,
                headers = {'content-type': 'image/tiff'})

    with open(Path(download_path) / filename, 'wb') as src:
        src.write(respb.content)

    print(f"File downloaded to {Path(download_path) / filename}")
    

In [7]:
download_asset(items[5])

Authenticating with netrc...
200


In [10]:
download_asset(items[7], asset_type='mask')

Authenticating with netrc...
200
